In [0]:
display(dbutils.fs.ls("/FileStore/"))

In [0]:
dbutils.fs.mkdirs("/FileStore/tables/arquivos_curso")

#TRABALHANDO COM ARQUIVOS DO IBGE


In [0]:
#LENDO ARQUIVO JSON
df = spark.read.json("/FileStore/tables/arquivos_curso/PNSB.json")
display(df)

In [0]:
#renomeando colunas
df = df.withColumnRenamed("D1C","cod_regiao")
df = df.withColumnRenamed("D1N","regiao")
df = df.withColumnRenamed("D2C","cod_variavel")
df = df.withColumnRenamed("D2N","variavel")
df = df.withColumnRenamed("D3C","cod_ano")
df = df.withColumnRenamed("D3N","ano")
df = df.withColumnRenamed("D4C","cod_doenca")
df = df.withColumnRenamed("D4N","doenca")
df = df.withColumnRenamed("MC","cod_medida")
df = df.withColumnRenamed("MN","medida")
df = df.withColumnRenamed("NC","cod_nivel_territorial")
df = df.withColumnRenamed("NN","nivel_territorial")
df = df.withColumnRenamed("V","valor")

In [0]:
#removendo primeira linha (header)
df = df.filter(df.valor!="Valor")
display (df)

In [0]:
#verificando Tipagem das colunas
df.printSchema()

In [0]:
#alterando tipagem das colunas
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

df_new = (
    df.withColumn("cod_regiao", col("cod_regiao").cast(IntegerType()))
    .withColumn("cod_variavel", col("cod_variavel").cast(IntegerType()))
    .withColumn("cod_ano", col("cod_ano").cast(IntegerType()))
    .withColumn("ano", col("ano").cast(IntegerType()))
    .withColumn("cod_doenca", col("cod_doenca").cast(IntegerType()))
    .withColumn("cod_medida", col("cod_medida").cast(IntegerType()))
    .withColumn("cod_nivel_territorial", col("cod_nivel_territorial").cast(IntegerType()))
    .withColumn("valor", col("valor").cast(IntegerType()))
)

##Comprimindo um arquivo JSON

In [0]:
#localizando o arquivo e vendo seu tamanho (size 129046)
display(dbutils.fs.ls("/FileStore/tables/arquivos_curso"))

In [0]:
#comprimindo o arquivo
df_new = (df.write.option("compression","gzip")
          .mode("overwrite")
          .format("json")
          .save("/FileStore/tables/arquivos_curso/json_gzip/")
)

In [0]:
#verificando compressão (size 4220)
display(dbutils.fs.ls("/FileStore/tables/arquivos_curso/json_gzip/"))

In [0]:
#lendo o arquivo comprimido
df = spark.read.option("compression","gzip").json("/FileStore/tables/arquivos_curso/json_gzip/")

display(df)

##Salvando dataframe em outro formato (csv)

In [0]:
#escrevendo um novo arquivo e salvando como csv
df.write.option("sep", ",").format("csv").save("/FileStore/tables/arquivos_curso/pnsb_csv/") 

In [0]:
#verificando arquivo
display(dbutils.fs.ls("/FileStore/tables/arquivos_curso/pnsb_csv/"))

#DESAFIO - QUAL MELHOR COMPRESSÃO PARA O ARQUIVO TRABALHADO

In [0]:
#documentação - https://spark.apache.org/docs/3.3.1/sql-data-sources-json.html
def salvando_arquivo_comprimido_json(dataframe):

  compressoes = ["none", "bzip2", "gzip", "lz4", "snappy", "deflate"]
  caminho_padrao = "/FileStore/tables/arquivos_curso/desafio/json_"

  for tipo_compressao in compressoes:
    dataframe.write\
      .option("compression", tipo_compressao)\
      .oprtion("header", true)
      .mode("overwrite") \
      .format("json") \
      .save(caminho_padrao + tipo_compressao)

    display(dbutils.fs.ls(caminho_padrao + tipo_compressao))

salvando_arquivo_comprimido_json(df)